In [4]:
import pandas as pd
import sqlite3


In [5]:
# Load the original data from the provided Excel file
original_file_path = "Copy Original Sample - Superstore.xls"
original_df = pd.read_excel(original_file_path, sheet_name="Orders")

# Extract the Year from the Order Date
original_df['Year'] = original_df['Order Date'].dt.year

# Calculate the total sales for 2018
sales_2018 = original_df[original_df['Year'] == 2018]['Sales'].sum()

# Let's see the result
sales_2018

484247.4981

In [6]:
# Connect to SQLite database
conn = sqlite3.connect("superstore.db")

# Load the original Superstore data
original_df = pd.read_excel("Copy Original Sample - Superstore.xls", sheet_name='Orders')
returns_df = pd.read_excel("Copy Original Sample - Superstore.xls", sheet_name='Returns')

# Merge the returns data with the main dataset
merged_df = original_df.merge(returns_df, how='left', left_on='Order ID', right_on='Order ID')
merged_df['Returned'] = merged_df['Returned'].fillna('No')

# Extract the Year and Quarter from the Order Date
merged_df['Year'] = merged_df['Order Date'].dt.year
merged_df['Quarter'] = merged_df['Order Date'].dt.quarter

In [7]:
# Write the updated data to SQLite
merged_df.to_sql('superstore_merged', conn, index=False, if_exists='replace')


/Users/kei/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:2779: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [8]:
# Extract the Year from the Order Date
merged_df['Year'] = merged_df['Order Date'].dt.year

# Group by City and calculate sum metrics
city_grouped = merged_df.groupby(['Year', 'Region', 'State', 'City', 'Category', 'Sub-Category']).agg({
    'Sales': 'sum',
    'Quantity': 'sum',
    'Discount': 'mean',  # Assuming you want an average discount for the city
    'Profit': 'sum',
    'Returned': lambda x: (x == 'Yes').sum()  # Count the number of 'Yes' in Returned column
}).reset_index()

# Rename columns for clarity
city_grouped = city_grouped.rename(columns={
    'Sales': 'Sales per City',
    'Quantity': 'Units Sold per City',
    'Returned': 'Returns per City'
})

# Calculate the Gross Profit Margin
city_grouped['Gross Profit Margin'] = (city_grouped['Profit'] / city_grouped['Sales per City']) * 100
city_grouped['Discount Rate'] = city_grouped['Discount'] * 100  # Convert discount to a percentage

# Reorganizing the dataframe as per the required columns
organized_df = city_grouped[['Year', 'Region', 'State', 'City', 'Category', 'Sub-Category', 
                             'Sales per City', 'Units Sold per City', 'Returns per City', 
                             'Discount Rate', 'Gross Profit Margin']]

organized_df


,Year,Region,State,City,Category,Sub-Category,Sales per City,Units Sold per City,Returns per City,Discount Rate,Gross Profit Margin
0,2018,Central,Illinois,Aurora,Furniture,Tables,268.935,3,0,50.0,-78.000000
1,2018,Central,Illinois,Aurora,Office Supplies,Binders,29.932,7,0,80.0,-155.000000
2,2018,Central,Illinois,Aurora,Technology,Phones,38.272,4,0,20.0,10.000000
3,2018,Central,Illinois,Bloomington,Furniture,Tables,617.700,6,0,50.0,-66.000000
4,2018,Central,Illinois,Bolingbrook,Office Supplies,Binders,9.980,5,0,80.0,-165.000000
...,...,...,...,...,...,...,...,...,...,...,...
4987,2021,West,Washington,Seattle,Technology,Phones,2727.312,74,11,20.0,6.543945
4988,2021,West,Washington,Spokane,Furniture,Chairs,1139.136,4,2,20.0,1.250000
4989,2021,West,Washington,Spokane,Furniture,Tables,70.980,1,1,0.0,29.000000
4990,2021,West,Washington,Spokane,Office Supplies,Storage,435.720,23,2,0.0,21.181906


In [9]:
organized_df.to_csv("organized_superstore_data.csv", index=False)


In [12]:
print(organized_df.columns)


Index(['Year', 'Region', 'State', 'City', 'Category', 'Sub-Category',
       'Sales per City', 'Units Sold per City', 'Returns per City',
       'Discount Rate', 'Gross Profit Margin'],
      dtype='object')


In [13]:
# Convert discount rate to discount value
organized_df['Discount Value'] = organized_df['Sales per City'] * organized_df['Discount Rate']

# Group by columns and aggregate accordingly
aggregated_df = organized_df.groupby(['Year', 'Region', 'State', 'City', 'Category']).agg({
    'Sales per City': 'sum',
    'Units Sold per City': 'sum',
    'Returns per City': 'sum',
    'Discount Value': 'sum',
    'Gross Profit Margin': 'mean'  # assuming you want the average gross profit margin
}).reset_index()

# Since we're aggregating, the column names will be changed to be more appropriate
aggregated_df.rename(columns={
    'Sales per City': 'Total Sales',
    'Units Sold per City': 'Total Units Sold',
    'Returns per City': 'Total Returns',
    'Discount Value': 'Total Discount'
}, inplace=True)

aggregated_df.head()


<ipython-input-13-51d1571cfce7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  organized_df['Discount Value'] = organized_df['Sales per City'] * organized_df['Discount Rate']


,Year,Region,State,City,Category,Total Sales,Total Units Sold,Total Returns,Total Discount,Gross Profit Margin
0,2018,Central,Illinois,Aurora,Furniture,268.935,3,0,13446.75,-78.0
1,2018,Central,Illinois,Aurora,Office Supplies,29.932,7,0,2394.56,-155.0
2,2018,Central,Illinois,Aurora,Technology,38.272,4,0,765.44,10.0
3,2018,Central,Illinois,Bloomington,Furniture,617.700,6,0,30885.00,-66.0
4,2018,Central,Illinois,Bolingbrook,Office Supplies,9.980,5,0,798.40,-165.0


In [14]:
aggregated_df.to_csv("aggregated_superstore_data.csv", index=False)


In [15]:
import pandas as pd
import sqlite3

# 1. Load the CSV file into a DataFrame
file_path = "aggregated_superstore_data.csv"
df = pd.read_csv(file_path)

# 2. Extract the top 20% based on the "Gross Profit Margin" column
threshold = df['Gross Profit Margin'].quantile(0.80)  # 80th percentile
top_20_percent_profit = df[df['Gross Profit Margin'] >= threshold]

threshold_loss = df['Gross Profit Margin'].quantile(0.20)  # 20th percentile
top_20_percent_loss = df[df['Gross Profit Margin'] <= threshold_loss]

top_20_percent_df = pd.concat([top_20_percent_profit, top_20_percent_loss])

# 3. Store the filtered data in SQLite
conn = sqlite3.connect("superstore_analysis.db")
top_20_percent_df.to_sql("top_20_percent_data", conn, if_exists="replace", index=False)

# You can now run SQL queries on the "top_20_percent_data" table in the "superstore_analysis.db" SQLite database.


/Users/kei/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:2779: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [17]:
# Establish a connection to the SQLite database
conn = sqlite3.connect("superstore_analysis.db")

# Fetch data from the top_20_percent_data table
query = "SELECT * FROM top_20_percent_data"
result_df = pd.read_sql(query, conn)

# Display the result
print(result_df)

# Close the connection
conn.close()



      Year   Region    State         City         Category  Total Sales  \
0     2018  Central  Indiana  Bloomington  Office Supplies      341.000   
1     2018  Central  Indiana     Columbus  Office Supplies       75.880   
2     2018  Central  Indiana     Columbus       Technology        1.980   
3     2018  Central  Indiana      Portage  Office Supplies       16.280   
4     2018  Central  Indiana     Richmond  Office Supplies      271.260   
...    ...      ...      ...          ...              ...          ...   
1036  2021     West   Oregon    Hillsboro  Office Supplies       23.766   
1037  2021     West   Oregon     Portland  Office Supplies      124.402   
1038  2021     West   Oregon      Redmond        Furniture      177.225   
1039  2021     West   Oregon      Redmond  Office Supplies       88.074   
1040  2021     West   Oregon  Springfield        Furniture     1178.048   

      Total Units Sold  Total Returns  Total Discount  Gross Profit Margin  
0                   25

In [19]:
# Let's take a look at the first few rows of the aggregated dataset to understand its structure
aggregated_df.head()


,Year,Region,State,City,Category,Total Sales,Total Units Sold,Total Returns,Total Discount,Gross Profit Margin
0,2018,Central,Illinois,Aurora,Furniture,268.935,3,0,13446.75,-78.0
1,2018,Central,Illinois,Aurora,Office Supplies,29.932,7,0,2394.56,-155.0
2,2018,Central,Illinois,Aurora,Technology,38.272,4,0,765.44,10.0
3,2018,Central,Illinois,Bloomington,Furniture,617.700,6,0,30885.00,-66.0
4,2018,Central,Illinois,Bolingbrook,Office Supplies,9.980,5,0,798.40,-165.0


In [20]:
import pandas as pd
import sqlite3

# Load the aggregated data
aggregated_df = pd.read_csv("aggregated_superstore_data.csv")

# Connect to the SQLite database
conn = sqlite3.connect("superstore_data.db")

# Load the aggregated data into the database
aggregated_df.to_sql("aggregated_data", conn, if_exists="replace", index=False)

# Query for the top 3 cities with the highest profit
top_profit_cities_query = """
SELECT City, SUM(`Total Sales`) as Total_Profit
FROM aggregated_data
GROUP BY City
ORDER BY Total_Profit DESC
LIMIT 3;
"""
top_profit_cities = pd.read_sql(top_profit_cities_query, conn)

# Query for the top 3 cities with the lowest profit
bottom_profit_cities_query = """
SELECT City, SUM(`Total Sales`) as Total_Profit
FROM aggregated_data
GROUP BY City
ORDER BY Total_Profit ASC
LIMIT 3;
"""
bottom_profit_cities = pd.read_sql(bottom_profit_cities_query, conn)

# Query for the top 3 cities with the highest discount
top_discount_cities_query = """
SELECT City, SUM(`Total Discount`) as Total_Discount
FROM aggregated_data
GROUP BY City
ORDER BY Total_Discount DESC
LIMIT 3;
"""
top_discount_cities = pd.read_sql(top_discount_cities_query, conn)

# Query for the top 3 cities with the highest returns
top_returns_cities_query = """
SELECT City, SUM(`Total Returns`) as Total_Returns
FROM aggregated_data
GROUP BY City
ORDER BY Total_Returns DESC
LIMIT 3;
"""
top_returns_cities = pd.read_sql(top_returns_cities_query, conn)

# Display the results
top_profit_cities, bottom_profit_cities, top_discount_cities, top_returns_cities



/Users/kei/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:2779: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


(            City  Total_Profit
 0  New York City   401212.8050
 1    Los Angeles   237140.2785
 2        Seattle   164998.7520,
       City  Total_Profit
 0  Abilene         1.392
 1   Elyria         1.824
 2  Jupiter         2.064,
             City  Total_Discount
 0   Philadelphia    4.278564e+06
 1  New York City    3.125156e+06
 2    Los Angeles    2.983141e+06,
             City  Total_Returns
 0    Los Angeles            375
 1  New York City            321
 2  San Francisco            238)

In [21]:
# List of cities to filter
cities_to_include = ["New York City", "Los Angeles", "Seattle", "Abilene", "Elyria", "Jupiter", "Philadelphia", "San Francisco"]

# Filter the dataframe
filtered_df = aggregated_df[aggregated_df['City'].isin(cities_to_include)]

# Export to CSV
filtered_df.to_csv("selected_cities_data.csv", index=False)


In [22]:
# List of cities to filter
cities_to_include = ["New York City", "Los Angeles", "Seattle", "Abilene", "Elyria", "Jupiter", "Philadelphia", "San Francisco"]

# Filter the dataframe for year 2021 and the specified cities
filtered_df_2021 = aggregated_df[(aggregated_df['Year'] == 2021) & (aggregated_df['City'].isin(cities_to_include))]

# Export to CSV
filtered_df_2021.to_csv("selected_cities_data_2021.csv", index=False)

In [26]:
import pandas as pd
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect("superstore.db")

# Load the data into the database
aggregated_df = pd.read_csv("selected_cities_data_2021.csv")
aggregated_df.to_sql("aggregated_data_2021", conn, if_exists="replace", index=False)

# Queries to get the top and bottom cities based on various metrics
queries = {
    "Total_Profit": "SELECT City, SUM(`Total Sales`) as Value FROM aggregated_data_2021 WHERE Year = 2021 GROUP BY City ORDER BY Value",
    "Total_Discount": "SELECT City, SUM(`Total Discount`) as Value FROM aggregated_data_2021 WHERE Year = 2021 GROUP BY City ORDER BY Value",
    "Total_Returns": "SELECT City, SUM(`Total Returns`) as Value FROM aggregated_data_2021 WHERE Year = 2021 GROUP BY City ORDER BY Value DESC"
}

# Execute queries and get the results
top_profit = pd.read_sql(queries["Total_Profit"] + " DESC LIMIT 3", conn)
top_loss = pd.read_sql(queries["Total_Profit"] + " ASC LIMIT 3", conn)
top_discount = pd.read_sql(queries["Total_Discount"] + " DESC LIMIT 3", conn)
top_return = pd.read_sql(queries["Total_Returns"] + " LIMIT 3", conn)

# Rearrange the rows
result = pd.concat([top_profit, top_loss, top_discount, top_return])

# Save to CSV
result.to_csv("rearranged_cities_data.csv", index=False)
